In [49]:
import json
import pandas as pd
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

# Load your labeled dataset (gold labels)
with open('tokenized_ner_data_6.json', 'r') as f:
    data = json.load(f)

# Convert to a DataFrame for easier handling
df = pd.DataFrame(data)

In [50]:
# Load the pre-trained NER model and tokenizer
model_name = "ner_model_2"
#"dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")

Device set to use cuda:0


In [51]:
label_mapping = {
    'MAL-ORG': 'I-ORG',  # Map MAL-ORG to I-ORG
    'ORG': 'I-ORG',       # Map ORG to I-ORG
    'PER': 'I-PER',       # Map PERSON entities to I-PER
    'LOC': 'I-LOC',       # Map LOCATION entities to I-LOC
    'Event': 'I-MISC',    # Map Event-related entities to I-MISC
    'Software': 'I-MISC', # Map Software-related entities to I-MISC
    'CVE': 'I-MISC',      # Map CVE to I-MISC
    'Malware': 'I-MISC',  # Map Malware to I-MISC
    'MISC': 'I-MISC',      # General MISC category to I-MISC
}

In [52]:
def map_labels(labels, label_mapping):
    """
    Map domain-specific labels to generic labels using the provided mapping.
    """
    mapped_labels = []
    for label in labels:
        if label == "O":  # Keep "O" as is
            mapped_labels.append(label)
        else:
            # Extract the prefix (B- or I-) and the entity type
            prefix = label[:2]  # "B-" or "I-"
            entity_type = label[2:]  # The rest of the label

            # Map the entity type using the mapping
            mapped_entity = label_mapping.get(entity_type, "O")  # Default to "O" if not in mapping

            # Combine the prefix with the mapped entity type only once
            mapped_labels.append(f"{prefix}{mapped_entity.lstrip('B-').lstrip('I-')}")
    return mapped_labels

In [ ]:
# Predict labels for each text in the dataset
def predict_labels(tokens):
    # Decode tokens to reconstruct the original text
    text = tokenizer.decode(tokenizer.convert_tokens_to_ids(tokens), skip_special_tokens=True)
    
    # Get model predictions using the NER pipeline
    predictions = ner_pipeline(text)
    
    # Initialize the predicted labels with "O"
    predicted_labels = ["O"] * len(tokens)
    
    for pred in predictions:
        pred_text = pred['word']
        pred_start, pred_end = pred['start'], pred['end']

        # Use 'entity_group' instead of 'entity'
        entity_label = pred['entity_group']
        
        # Encode the predicted text to match token positions
        encoded_pred = tokenizer.encode(pred_text, add_special_tokens=False)
        
        # Match the predicted span with tokens
        for i, token_id in enumerate(tokenizer.convert_tokens_to_ids(tokens)):
            token_start = text.find(tokenizer.decode([token_id], skip_special_tokens=True), pred_start)
            token_end = token_start + len(tokenizer.decode([token_id], skip_special_tokens=True))
            
            if token_start == pred_start and token_end == pred_end:
                predicted_labels[i] = entity_label #pred['entity']
    
    return predicted_labels

# Apply predictions to each row
df['predicted_labels'] = df['tokens'].apply(predict_labels)

In [55]:
# Apply the label mapping to the gold labels
df['mapped_labels'] = df['labels'].apply(lambda x: map_labels(x, label_mapping))

In [56]:
# Flatten gold labels and predictions for evaluation
all_gold = [label for labels in df['mapped_labels'] for label in labels]
all_pred = [pred for preds in df['predicted_labels'] for pred in preds]

# Generate a classification report
print(classification_report(all_gold, all_pred))

              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00       185
      B-MISC       0.00      0.00      0.00       527
       B-ORG       0.00      0.00      0.00       735
       B-PER       0.00      0.00      0.00        85
         CVE       0.00      0.00      0.00         0
       Event       0.00      0.00      0.00         0
       I-LOC       0.00      0.00      0.00       183
      I-MISC       0.00      0.00      0.00      2139
       I-ORG       0.00      0.00      0.00      1903
       I-PER       0.00      0.00      0.00       265
         LOC       0.00      0.00      0.00         0
     MAL-ORG       0.00      0.00      0.00         0
        MISC       0.00      0.00      0.00         0
           O       0.82      0.99      0.90     24857
         ORG       0.00      0.00      0.00         0
         PER       0.00      0.00      0.00         0
    Software       0.00      0.00      0.00         0

    accuracy              

c:\Users\magnu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\magnu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\magnu\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

In [57]:
def inspect_predictions(df, num_examples=5):
    """
    Inspect predicted labels, gold labels, and tokens for a sample of examples.
    :param df: The dataframe containing 'tokens', 'mapped_gold_labels', and 'mapped_predictions'.
    :param num_examples: Number of examples to display.
    """
    for i in range(num_examples):
        print(f"Example {i + 1}:")
        print("-" * 40)
        print("Tokens:            ", "\t\t".join(df.loc[i, 'tokens']))
        print("Gold Labels:       ", "\t\t".join(df.loc[i, 'mapped_labels']))
        print("Predicted Labels:  ", "\t\t".join(df.loc[i, 'predicted_labels']))
        print("\n")

# Inspect a few examples
inspect_predictions(df, num_examples=5)

Example 1:
----------------------------------------
Tokens:             [CLS]		A		new		ransom		##ware		-		as		-		a		-		service		(		Ra		##a		##S		)		operation		named		C		##ica		##da		##33		##01		has		already		listed		19		victims		on		its		ex		##tor		##tion		portal		,		as		it		quickly		attacked		companies		worldwide		.		The		new		c		##y		##ber		##c		##rim		##e		operation		is		named		after		the		mysterious		2012		-		2014		online		/		real		-		world		game		that		involved		elaborate		cry		##pt		##ographic		puzzles		and		used		the		same		logo		for		promotion		on		c		##y		##ber		##c		##rim		##e		forums		.		However		,		there		'		s		no		connection		between		the		two		,		and		the		legitimate		project		has		issued		a		statement		to		re		##nounce		any		association		and		con		##de		##m		##n		the		ransom		##ware		operators		'		actions		.		The		C		##ica		##da		##33		##01		Ra		##a		##S		first		began		promoting		the		operation		and		recruiting		affiliates		on		June		29		,		202		##4		,		in		a		forum		pos